In [151]:
"""
Проверка наличия данных из ведомости лесов в реестре УПП с отображением
"""
import pandas as pd
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows
import time
pd.options.mode.chained_assignment = None  # default='warn'

In [152]:
def convert_columns_to_str(df,number_columns):
    """
    Функция для конвертации указанных столбцов в строковый тип и очистки от пробельных символов в начале и конце
    """
    
    for column in number_columns: # Перебираем список нужных колонок
        df.iloc[:,column] = df.iloc[:,column].astype(str)
        # Очищаем колонку от пробельных символов с начала и конца
        df.iloc[:,column] = df.iloc[:,column].apply(lambda x: x.strip())


In [153]:
file_params = 'params.xlsx'
path_to_end_folder = 'data/'

In [154]:
params = pd.read_excel(file_params,header=None)

In [155]:
# Преврашаем каждую колонку в список
params_first_columns = params[0].tolist()
params_second_columns = params[1].tolist()



# Конвертируем в инт заодно проверяя корректность введенных данных
try:
    int_params_first_columns = list(map(int,params_first_columns))
    int_params_second_columns = list(map(int,params_second_columns))
except:
    print('В колонках файла параметров должны быть только цифры!!!')
    


# Отнимаем 1 от каждого значения чтобы привести к питоновским индексам
int_params_first_columns = list(map(lambda x:x-1,int_params_first_columns))
int_params_second_columns = list(map(lambda x:x-1,int_params_second_columns))

In [156]:
# Считываем из файлов только те колонки по которым будет вестись сравнение
first_df = pd.read_excel('data/Сравнение УПП с другими ведомостями на наличие участков или их отсутствие/2022-10-27_64_Реестр УПП с дополнительными колонками..xlsx',
                         skiprows=6,usecols=int_params_first_columns,keep_default_na=False)
second_df = pd.read_excel('data/Сравнение УПП с другими ведомостями на наличие участков или их отсутствие/Ведомость.xlsx',usecols=int_params_second_columns)
# Считываем файл параметров без заголовка, чем проще тем лучше


C:\Users\1\anaconda3\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [157]:
first_df.head()

,Лесничество,Участковое лесничество,Урочище,Номер лесного квартала,Номер лесотаксационного выдела
0,,,,,
1,1,2,3,4,5
2,Ангоянское,Ангоянское,,32,1
3,Ангоянское,Ангоянское,,32,2
4,Ангоянское,Ангоянское,,32,3


In [158]:
# Конвертируем нужные нам колонки в str
convert_columns_to_str(first_df,int_params_first_columns)
convert_columns_to_str(second_df,int_params_second_columns)

# Создаем в каждом датафрейме колонку с айди путем склеивания всех нужных колонок в одну строку
first_df['ID'] = first_df.iloc[:,int_params_first_columns].sum(axis=1)
second_df['ID'] = second_df.iloc[:,int_params_second_columns].sum(axis=1)




In [159]:
first_df['ID']



0                                
1                           12345
2         АнгоянскоеАнгоянское321
3         АнгоянскоеАнгоянское322
4         АнгоянскоеАнгоянское323
                   ...           
155845                           
155846                           
155847                           
155848                           
155849                           
Name: ID, Length: 155850, dtype: object

In [160]:
second_df['ID']

0                               ЗакаменскоеСанагинское1275
1                               ЗакаменскоеСанагинское1273
2                              ЗакаменскоеСанагинское12711
3                              ЗакаменскоеСанагинское12711
4                               ЗакаменскоеСанагинское1268
                               ...                        
26234    ЗакаменскоеЗакаменское сельскоеТехнический уча...
26235    ЗакаменскоеЗакаменское сельскоеТехнический уча...
26236    ЗакаменскоеЗакаменское сельскоеТехнический уча...
26237    ЗакаменскоеЗакаменское сельскоеТехнический уча...
26238    ЗакаменскоеЗакаменское сельскоеТехнический уча...
Name: ID, Length: 26239, dtype: object

In [161]:
first_df.shape

(155850, 6)

In [162]:
second_df.shape

(26239, 6)

In [163]:
# Обрабатываем дубликаты


first_df.drop_duplicates(subset=['ID'],keep='last',inplace=True)# Удаляем дубликаты из датафрейма


second_df.drop_duplicates(subset=['ID'],keep='last',inplace=True)# Удаляем дубликаты из датафрейма


In [164]:
first_df.shape

(89924, 6)

In [165]:
second_df.shape

(7991, 6)

In [166]:
# Создаем документ
wb = openpyxl.Workbook()
# создаем листы
ren_sheet = wb['Sheet']
ren_sheet.title = 'Итог'


In [167]:
# Создаем датафрейм
itog_df = pd.merge(first_df, second_df, how='outer', left_on=['ID'], right_on=['ID'],
                   indicator=True)


In [168]:
itog_df[itog_df['_merge'] == 'both']

,Лесничество_x,Участковое лесничество_x,Урочище,Номер лесного квартала,Номер лесотаксационного выдела,ID,Лесничество_y,Участковое лесничество_y,Урочище,Квартал,Выдел,_merge
31387,Закаменское,Баянгольское,,125,26,ЗакаменскоеБаянгольское12526,Закаменское,Баянгольское,,125,26,both
31388,Закаменское,Баянгольское,,125,27,ЗакаменскоеБаянгольское12527,Закаменское,Баянгольское,,125,27,both
31389,Закаменское,Баянгольское,,125,28,ЗакаменскоеБаянгольское12528,Закаменское,Баянгольское,,125,28,both
31398,Закаменское,Баянгольское,,142,3,ЗакаменскоеБаянгольское1423,Закаменское,Баянгольское,,142,3,both
31401,Закаменское,Баянгольское,,219,6,ЗакаменскоеБаянгольское2196,Закаменское,Баянгольское,,219,6,both
...,...,...,...,...,...,...,...,...,...,...,...,...
31977,Закаменское,Шара-Азаргинское,,106,2,ЗакаменскоеШара-Азаргинское1062,Закаменское,Шара-Азаргинское,,106,2,both
31978,Закаменское,Шара-Азаргинское,,106,3,ЗакаменскоеШара-Азаргинское1063,Закаменское,Шара-Азаргинское,,106,3,both
31979,Закаменское,Шара-Азаргинское,,106,4,ЗакаменскоеШара-Азаргинское1064,Закаменское,Шара-Азаргинское,,106,4,both
31982,Закаменское,Шара-Азаргинское,,138,2,ЗакаменскоеШара-Азаргинское1382,Закаменское,Шара-Азаргинское,,138,2,both


In [169]:
# Отфильтровываем значения both,right
out_df = itog_df[(itog_df['_merge'] == 'both') | (itog_df['_merge']=='right_only')]

In [170]:
out_df.rename(columns={'_merge':'Присутствие в реестре УПП'},inplace=True)

In [171]:
out_df['Присутствие в реестре УПП'] = out_df['Присутствие в реестре УПП'].apply(lambda x:'Имеется в реестре' if x == 'both' else 'Отсутствует в реестре')

In [172]:
out_df.to_excel('Тест.xlsx',index=False)